In [1]:
import scipy.io
import configparser
import datetime
import gc
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from typing import Any, Dict
import cv2
import h5py
import numpy as np
import pandas as pd
import scipy.io
import skimage.io as skio
import typing_extensions
from dateutil import tz
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.data_utils import DataChunkIterator
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, \
    MultiChannelVolume, MultiChannelVolumeSeries, SegmentationLabels
from pynwb import NWBFile, NWBHDF5IO
from pynwb.behavior import SpatialSeries, Position, BehavioralTimeSeries, BehavioralEvents
from pynwb.ophys import ImageSegmentation, PlaneSegmentation, \
    DfOverF, RoiResponseSeries, Fluorescence
from pynwb.image import ImageSeries
from tifffile import TiffFile
from tifffile import imread
from tqdm import tqdm
import nd2reader
import nrrd
from pathlib import Path

from support_library.nwb.validation import validate
from support_library.nwb.visualizer import visualize
from support_library.nwb.fixer import fix_subject, fix_track

In [2]:
def iterate_over_directory(path):
    ttl = 0
    for root, dirs, files in os.walk(path):        
        for file in files:
            if ".nwb" in file and "updated" not in file:
                ttl += 1
                filepath = os.path.join(root, file)
                validation_summary = ''
                    
                with NWBHDF5IO(filepath, mode='r') as read_io:
                    nwbfile = read_io.read()

                    is_valid, validation_summary = validate_file(filepath, nwbfile)

                    if ttl in subjects_to_visualize:
                        visualize_file(filepath, nwbfile)

                if not is_valid:
                    update_file(filepath, validation_summary)
                    
                    
def validate_file(filepath=None, nwbfile=None):
    if filepath is None:
        substring = ""
    else:
        substring = f" {os.path.basename(filepath)}"
        
    if nwbfile is None:
        nwbfile = NWBHDF5IO(filepath, mode='r').read()
        
    print(f" | Validating{substring}...")
    is_valid, validation_summary = validate(nwbfile)
    print(f" | Validation {'PASSED' if is_valid else 'FAILED'}:")
    print(f" | | " + validation_summary.replace('\n', f'\n | |'))
    
    return is_valid, validation_summary

def visualize_file(filepath=None, nwbfile=None):
    if filepath is None:
        substring = ""
    else:
        substring = f" {os.path.basename(filepath)}"
        
    if nwbfile is None:
        nwbfile = NWBHDF5IO(filepath, mode='r').read()
    
    print(f" | Visualizing{substring}...")
    visualize(nwbfile)

def update_file(filepath=None, validation_summary=None):          
    with NWBHDF5IO(filepath, mode='r') as read_io:
        nwbfile = read_io.read()
        og_id = nwbfile.subject.subject_id
        og_description = nwbfile.subject.description
        
    if "Strain: not specified" in validation_summary:
        fix_subject(filepath, strain='OH15500', description=og_description.replace('[]', ''))
    
    fix_track(filepath, reference='maedeh')      

In [3]:
path = 'E:\\0-WORK\\1-UMASSCHAN\\Data\\venkatachalam\\000981'
subjects_to_visualize = []

iterate_over_directory(path)

C:\Users\Kevin\AppData\Local\Programs\Python\Python310\lib\site-packages\hdmf\spec\namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


 | Validating sub-20220327-h2_ses-20220327_ophys.nwb...
 | Validation FAILED:
 | | Subject: PASSED
 | | Acquisitions: PASSED
 | | .processing Modules: PASSED
 | Validating sub-20220327-h4_ses-20220327_ophys.nwb...
 | Validation FAILED:
 | | Subject: PASSED
 | | Acquisitions: PASSED
 | | .processing Modules: PASSED
 | Validating sub-20220329-m10_ses-20220329_ophys.nwb...
 | Validation FAILED:
 | | Subject: PASSED
 | | Acquisitions: PASSED
 | | .processing Modules: PASSED


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'E:\0-WORK\1-UMASSCHAN\Data\venkatachalam\maedeh_onedrive\20220329\hermaphrodite\1\annotations.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
"""

tp = "E:\\0-WORK\\1-UMASSCHAN\\Data\\venkatachalam\\000981\\sub-20220327-h2\\sub-20220327-h2_ses-20220327_ophys_updated.nwb"
    
with NWBHDF5IO(tp, mode='r') as read_io:
    nwbfile = read_io.read()
    print(nwbfile.processing['NeuroPAL']['TrackedNeurons'][115])

"""